In [ ]:
!pip install -U sentence-transformers

In [ ]:
import requests
import json

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

import requests
import json

sim_model_name="Sakil/sentence_similarity_semantic_search"
sim_model = SentenceTransformer(sim_model_name)

In [ ]:
def sentence_in(reddits):
    sub_rules = []
    sub_out = []
    num_rules = []
    for sub, rules in reddits.items():
        sub_out.append(sub)
        i = 0
        for rule in rules.values():
            sub_rules.append(rule["short_name"] + " " + rule["description"])
            i = i + 1
        num_rules.append(i)
    return sub_rules, sub_out, num_rules

def sub_rules_right():
    f_open = open("rightrules.json")
    reddits = json.load(f_open)
    data = sentence_in(reddits)
    return data

def sub_rules_left():
    f_open = open("leftrules.json")
    reddits = json.load(f_open)
    data = sentence_in(reddits)
    return data


data_l, subs_out_l, num_rules_l = sub_rules_left()
data_r, subs_out_r, num_rules_r = sub_rules_right()

In [ ]:
#Encode all sentences
embeddings_left = sim_model.encode(data_l)
embeddings_right = sim_model.encode(data_r)

#Compute cosine similarity between all pairs
cos_sim = util.cos_sim(embeddings_left, embeddings_right)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []

print(len(cos_sim))
print(len(cos_sim[0]))

In [ ]:
print(cos_sim)

In [ ]:
for rows_left in range(len(cos_sim)):
    for cols_right in range(len(cos_sim[0])):
        if cos_sim[rows_left][cols_right] > 0.9:
            rr = 0
            ii = 0
            jj = 0
            ll = 0
            sub_i = " "
            sub_j = " "
            for w in range(len(num_rules_l)):
                if rows_left >= ll - 1 and rows_left < ll + num_rules_l[w]:
                    sub_i = subs_out_l[ii]
                ll += num_rules_l[w]
                ii += 1
            for vv in range(len(num_rules_r)):
                if cols_right >= rr - 1 and cols_right < rr + num_rules_r[vv]:
                    sub_j = subs_out_r[jj]
                jj += 1
                rr += num_rules_r[vv]
            all_sentence_combinations.append([cos_sim[rows_left][cols_right], data_l[rows_left], data_r[cols_right], sub_i, sub_j])

In [ ]:
print(len(all_sentence_combinations))

In [ ]:
# for row in all_sentence_combinations:
#     print(row)

In [ ]:
all_sentence_combinations_over_no = []
for row in all_sentence_combinations:
    row_new = []
    row_new.append(row[0])
    row_new.append(row[1])
    row_new.append(row[2])
    row_new.append([row[3], row[4]])
    all_sentence_combinations_over_no.append(row_new)

all_sentence_combinations_over_no = sorted(all_sentence_combinations_over_no, key=lambda x: x[3], reverse=True)

for row in all_sentence_combinations_over_no:
    print(row[3], row)


In [ ]:
all_sentence_combinations_out = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for x in all_sentence_combinations_out[0:10]:
      print(x)


overlap_A = []
overlap_both = []
for row in all_sentence_combinations_out:
    overlap_A.append(row[3])
    overlap_A.append(row[4])
    overlap_both.append([row[3], row[4]])
print(overlap_A)
print(overlap_both)

In [ ]:
import pandas as pd

# declaring the list

count = pd.Series(overlap_A).value_counts()
print("Element Count")
print(count)

In [ ]:
count = pd.Series(overlap_both).value_counts()
print("Element Count")
print(count)

In [ ]:
make_unique = sorted(all_sentence_combinations_over_no, key=lambda x: x[3], reverse=True)

same = make_unique[0][3]
print(same)


In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
cat = sentiment_pipeline(data_l + data_r)

In [ ]:
pos = 0
neg = 0
for line in cat:
  if line['label'] == 'POSITIVE':
      pos += 1
  if line['label'] == 'NEGATIVE':
      neg += 1

print("pos: ", pos)
print("neg: ", neg)


In [ ]:
print("Postive Rules: ", pos)
print("Negative Rules ", neg)
print("Percentage Negative: ", neg/(pos + neg) * 100, "%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

graph_1 = []
graph_count = []
#counter = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1]
counter = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
for n in counter:
    tab = 0
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            if cos_sim[i][j] > n:
                tab += 1
    graph_count.append(tab)

xpoints = np.array(counter)
ypoints = np.array(graph_count)


plt.plot(xpoints, ypoints)
plt.show()

In [ ]:
print(graph_count)